# Address sanitizer demo
This demo runs on SWAN, but it requires a software stack with gcc10.

## Heap bugs
- Experiment with the command-line argument. Note that the bug often goes unnoticed when not using the sanitizer.

In [ ]:
%%file testAsan.cpp
#include <string>
#include <cstring>

const char* heapOverflow(char* mem, int i) {
  mem[i] = 'A';
  return mem;
}

void freeMem(char* mem) { delete mem; }
int main(int argc, char* argv[]) {
  auto mem = new char[8];
  memset(mem, '_', 8);
  strcpy(mem, "Hello");
  printf("%s\n", heapOverflow(mem, std::stoi(argv[1])));
  
//   freeMem(mem);
  return 0;
}

In [ ]:
%%shell
export ASAN_OPTIONS=detect_leaks=0
sed -e '1s/^/\n/' -i testAsan.cpp; g++ -fsanitize=address -g -std=c++17 testAsan.cpp;

In [ ]:
%%shell
./a.out 3

- Check out leak sanitizer when not in a notebook. Output:
        =================================================================
        ==19929==ERROR: LeakSanitizer: detected memory leaks

        Direct leak of 8 byte(s) in 1 object(s) allocated from:
            #0 0x7f84b0bf0577 in operator new[](unsigned long) /build/gcc/WORK/build/contrib/gcc-10.1.0/src/gcc/10.1.0/libsanitizer/asan/asan_new_delete.cpp:102
            #1 0x4024e7 in main /eos/home-s/shageboe/SWAN_projects/ASAN/testAsan.cpp:12
            #2 0x7f84b0463554 in __libc_start_main (/lib64/libc.so.6+0x22554)

        SUMMARY: AddressSanitizer: 8 byte(s) leaked in 1 allocation(s).
- Test deletion errors, e.g. a double delete.

- `valgrind` also finds these errors, but runs slower
- `valgrind` needs more arguments, e.g. `--track-origins=yes --leak-check=full`, slowing it down even more

## Stack bugs
- `valgrind` cannot survey the stack.
- **Note**: Try adding a variable to the stack frame.

In [ ]:
%%file testAsan.cpp
#include <string>
#include <cstring>

std::string stackOverflow(int i) {
  char chars[6] = "hello";
  chars[i] = 'A';
  return chars;
}

int main(int argc, char* argv[]) {
  const int arg = std::stoi(argv[1]);
  printf("%s", stackOverflow(arg).data());

  return 0;
}


In [ ]:
%%shell
sed -e '1s/^/\n/' -i testAsan.cpp; g++ -g -fsanitize=address -std=c++17 testAsan.cpp

In [ ]:
%%shell
./a.out 6

## Stack use after return
- `asan` has an extended mode that poisons the stack memory after `return`.
- ROOT had a few bugs like this.

In [ ]:
%%file testAsan.cpp
#include <string>
#include <cstring>

const char* stackUseAfterReturn() {
  std::string str = "hello";
  auto ptr = str.data(); // Confuse the compiler
  return ptr;
}

int main(int, char**) {
  printf("%s\n", stackUseAfterReturn());
  return 0;
}


In [ ]:
%%shell
sed -e '1s/^/\n/' -i testAsan.cpp; g++ -g -fsanitize=address -std=c++17 testAsan.cpp

It requires the run-time flag `detect_stack_use_after_return`:

In [ ]:
%%shell
ASAN_OPTIONS=detect_stack_use_after_return=1 ./a.out 0 
echo test